In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Create PyTorch Dataloader from object
# IMPORTANT: Make sure you are in highest directory (i.e. one directory above content)
import torch
import torchvision
import torch.utils.data as data
from torchvision import transforms
import PIL
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

myTransforms = transforms.Compose([
    transforms.ColorJitter(hue=.1, saturation=.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
!mkdir -p /monet/monet_jpg
!cp -r /kaggle/input/gan-getting-started/monet_jpg /monet/monet_jpg
DATA_PATH = "/monet"
BATCH_SIZE = 128

random.seed(20)

train_data = torchvision.datasets.ImageFolder(root=DATA_PATH, transform = myTransforms)
train_data = torch.utils.data.ConcatDataset([train_data] * 32)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Look at i'th image of dataset if you'd like (totally not necessary)
# May not work if you use a transform with train_data
from IPython.display import display # to display images
i = 0

pil_im = train_data[i][0]
display(pil_im)

In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torchvision
import torch.utils.data as data
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
# Root directory for dataset
dataroot = "~/torch_datasets"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer. MNIST size is actually 28x28 but it should be okay.
#image_size = 64 # For MNIST
image_size = 256 # For Monet

# Number of channels in the training images. For color images this is 3.
# MNIST is black and white, so 1.
#nc = 1 # For MNIST
nc = 3 # For Monet

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
#ngf = 64 # For MNIST
ngf = 32 # For Monet

# Size of feature maps in discriminator
#ndf = 64 # For MNIST
ndf = 32 # For Monet

# Number of training epochs
num_epochs = 50

# Learning rate for optimizers
lr = 0.0001

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
beta2 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:
transform = transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size), 
                                transforms.ToTensor()
                              ])
'''
dataset = dset.MNIST(
    root=dataroot, train=True, transform=transform, download=True
)
'''
dataset = train_data

loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True
)

len(loader.dataset)

# size of mnist image: [1, 64, 64]
# size of monet image: [3, 256, 256]

In [ ]:
examples = enumerate(loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
# set the device to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# visualizing the dataset.
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(12):
  plt.subplot(3,4,i+1)
  plt.tight_layout()
  # must permute to get shape from [3, 256, 256] -> [256, 256, 3]
  plt.imshow(example_data[i].permute(1,2,0), interpolation='none') #cmap = 'hsv' maybe
  plt.xticks([])
  plt.yticks([])

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Create the Generator

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 10, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 10, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 10, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # 
            nn.ConvTranspose2d( ngf * 2, ngf, 12, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 12, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Instantiate the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

In [ ]:
# Coding the discriminator
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 0, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 2, ndf * 4, 4, 4, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 4, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Initialize the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2)) # /2.4693701
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(loader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        print(data[0].shape)
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((data[0].shape[0],), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        print("before output")
        output = netD(real_cpu).view(-1) # Getting broken pipe error
        print("after ouput")
        print(output.size())
        # Calculate loss on all-real batch
        errD_real = criterion(output, label) # This line results in a size mismatch error when I run it with the monet dataset
        #ValueError: Target and input must have the same number of elements. target nelement (128) != input nelement (21632)
        # It appears that this is because it is bc the output is 13x13 instead of 1x1
        # Fixed this, now see broken pipe error
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        print(fake.detach().shape)
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        #if i % 50 == 0:
        if True:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(loader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 50 == 0) or ((epoch == num_epochs-1) and (i == len(loader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        if D_G_z1 == 0 and D_G_z2 == 0:
          epoch = num_epochs
          break

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#plt.imshow(img_list[-1].permute(1,2,0))
idk = netG(torch.randn(10, nz, 1, 1, device=device)).detach().cpu()
plt.imshow(np.transpose(idk[1],(1,2,0)))

In [ ]:
from zipfile import ZipFile
zipObj = ZipFile('images.zip', 'w')
for i in range(7000):
    img = netG(torch.randn(1, nz, 1, 1, device=device)).detach().cpu()[0]
    fp = 'img' + str(i) + '.jpg'
    torchvision.utils.save_image(img, fp)
    zipObj.write(fp)
zipObj.close()
!rm *.jpg
print("done")

In [ ]:
#animation of generator over time
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

#plt.imshow(img_list[1].permute(1,2,0))
#idk = netG(torch.randn(10, nz, 1, 1, device=device)).detach().cpu()
#plt.imshow(idk[0].permute(1,2,0))
